# Notkun gervigreindar fyrir teikningu þrívíddarmynda

Nathan HK

In [2]:
import numpy as np
import os
import pywikibot
import sklearn as sk
from sklearn.model_selection import train_test_split
from stl import mesh
import time
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
torch.set_default_device(device)
mappa = '/Users/002-nathan/Desktop/Envalys/envalys-nathan/'  # Change this

## Inngangsorð
Við ætlum að þjálfa gervigreindarlíkan til að teikna þrívíddarmyndir. Hvernig gerum við það?
- Fyrst reyndi ég að nota þrívíddardíla eða "þríla" (e. *voxels*). Ég taldi punktana í hverjum þríl, og þjálfaði líkan með tölunum, eins og líkan sem greinir ljósmyndir. GitHub: https://github.com/nholmesking/envalys-nathan/blob/4fd89a1515c1c3623a5b8cdf8c766858c465e451/Nathan%20HK.ipynb
  - Vandamál #1: Mac-GPU getur ekki neitt með þrívíddargögnum. Ég þurfti að nota CPU, sem er of hægt.
  - Vandamál #2: Hraðinn er $O(n^3)$.
  - Vandamál #3: Það var engin klár leið til að fara frá greiningu yfir á teikningu.
- Þá reyndi ég að nota líkanið 3DShape2VecSet, sem tekur lista yfir punkta og teinkir myndir sem eru eins og listinn.

Tilvísanir:
- https://arxiv.org/pdf/2301.11445
- https://github.com/1zb/3DShape2VecSet

Ég gerði litlar breytingar á upprunaforritið, því það er eitt forritasafn (e. *module*) sem virkar ekki lengur, og forritið var skrifað fyrir Windows (og ég nota Mac).

## Gögn
Gögnin frá 3DShape2VecSet eru 650 GB samtals; þetta er of mikið fyrir tölvuna mína.

Þessi gögn eru STL-skrár frá Wikimedia Commons. Það eru fimm flokkar:
- líkamshlutar
- byggingar
- rúmfræði
- geimfarartæki
- styttur

### Sækja gögn

In [3]:
flokkar = ['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
skrar = {}
catnum = {}

In [4]:
commons = pywikibot.Site('commons', 'commons')
cn = 0
for a in flokkar:
    print(a)
    cat = pywikibot.Category(commons, 'STL files of ' + a)
    catnum[a] = cn
    cn += 1
    n = 0
    for p in cat.members(member_type=['file']):
        if n % 10 == 0:
            print(n)
        mynd = pywikibot.FilePage(p)
        try:
            tempf = open(mappa + 'STLdata/' + a + '_' + p.title()[5:], 'r')
            tempf.close()
        except FileNotFoundError:
            mynd.download(filename=mappa + 'STLdata/' + a + '_' + p.title()[5:])
        try:
            skrar[a].append(p.title()[5:])
        except KeyError:
            skrar[a] = [p.title()[5:]]
        n += 1
        if n >= 100:
            break

body parts
0
10
20
30
40
50
60
70
80
buildings
0
10
20
geometric shapes
0
10
20
30
40
objects in space
0
10
20
30
40
50
sculptures
0
10
20
30
40
50


### Setja upp gögn fyrir notkun
Við eigum að breyta gögnunum úr STL-sniði í sniðið sem 3DShape2VecSet notar. Þetta snið er með lista yfir punkta eftir hvort þeir séu innan á forminu eða utan, en gögnin frá Wikimedia Commons er ekki með það.

Með ```isInside()``` getum útreiknað hvort lína og þríhyrningur mætast, og í hvora átt. Við búum til tvær línur sem fara beint í X-, Y-, eða Z-áttina frá punktnum (eina línu upp og eina línu niður), og teljum þríhyrningana sem línan mætir; ef talan er slétt, þá er punkturinn utan á forminu, og ef talan er oddatala, þá er punkturinn innan. En það tekur of langan tíma; sumar myndir eru með fleiri en milljón þríhyrninga. 

Þess vegna bjó ég til þetta reknirit: Við deilum myndinni í 32x32 eða fleiri ílát í þrem ílátahópum, einum fyrir hverja vídd, eftir hnitum hinna vídda. Í byrjun setjum við alla þríhyrninga í þeirra ílát; sumir þríhyrningar, sem eru á línunni milli íláta, eru settir í tvö eða fleiri ílát. Fyrir hvern listapunkt eigum við bara að leita í rétta ílátinu, í hópnum þar sem þríhyrningafjöldinn er lægstur.

Með þessu reikniriti er sniðbreytingin nærri **400** sinnum fljótari. Ég skrifaði sjálfur allt hérna nema ```isInside()```; sjáðu kóðann fyrir tilvísun.

In [14]:
def isInside(a, randpoint, att):
    if (a[0] > randpoint[0] and a[3] > randpoint[0] and a[6] > randpoint[0]):
        return 0
    if (a[0] < randpoint[0] and a[3] < randpoint[0] and a[6] < randpoint[0]):
        return 0
    if (a[1] > randpoint[1] and a[4] > randpoint[1] and a[7] > randpoint[1]):
        return 0
    if (a[1] < randpoint[1] and a[4] < randpoint[1] and a[7] < randpoint[1]):
        return 0
 
    # This part is from https://diegoinacio.github.io/creative-coding-notebooks-page/pages/ray-intersection_triangle.html
    if att == 'x':
        e = np.array([1, 0, 0])     # Ray direction
    elif att == 'y':
        e = np.array([0, 1, 0])     # Ray direction
    elif att == 'z':
        e = np.array([0, 0, 1])     # Ray direction
    AB = a[3:6] - a[0:3]               # Oriented segment A to B
    AC = a[6:9] - a[0:3]               # Oriented segment A to C
    n = np.cross(AB, AC)     # Normal vector
    n_ = n/np.linalg.norm(n) # Normalized normal
    if not np.any(np.dot(n_, e)):
        return 0
    d = - np.dot(n_, a[0:3])
    # Finding parameter t
    t = - (np.dot(n_, randpoint) + d)/np.dot(n_, e)
    P = randpoint + t*e
    # Get the resulting vector for each vertex
    # following the construction order
    Pa = np.dot(np.cross(a[3:6] - a[0:3], P - a[0:3]), n_)
    Pb = np.dot(np.cross(a[6:9] - a[3:6], P - a[3:6]), n_)
    Pc = np.dot(np.cross(a[0:3] - a[6:9], P - a[6:9]), n_)

    if t > 0 and (Pa > 0 and Pb > 0 and Pc > 0):
        return 1
        
    if att == 'x':
        e = np.array([-1, 0, 0])     # Ray direction
    elif att == 'y':
        e = np.array([0, -1, 0])     # Ray direction
    elif att == 'z':
        e = np.array([0, 0, -1])     # Ray direction
    # Finding parameter t
    t = - (np.dot(n_, randpoint) + d)/np.dot(n_, e)
    P = randpoint + t*e
    # Get the resulting vector for each vertex
    # following the construction order
    Pa = np.dot(np.cross(a[3:6] - a[0:3], P - a[0:3]), n_)
    Pb = np.dot(np.cross(a[6:9] - a[3:6], P - a[3:6]), n_)
    Pc = np.dot(np.cross(a[0:3] - a[6:9], P - a[6:9]), n_)

    if t > 0 and (Pa > 0 and Pb > 0 and Pc > 0):
        return -1
    return 0

In [17]:
for cat in skrar:
    tts = []
    print(cat)
    byrjun = time.time()
    os.system('mkdir "' + mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '"')
    os.system('mkdir "' + mappa + '3DShape2VecSet/data/ShapeNetV2_watertight/' + cat + '"')
    for fi in skrar[cat]:
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        utskra_mesh = open(mappa + '3DShape2VecSet/data/ShapeNetV2_watertight/' + cat + '/' + fi + '.npz', 'wb')
        np.savez(utskra_mesh, points=np.array(gogn.v0, dtype=np.float32), scale=1)
        tts.append(fi)
    train, val = train_test_split(tts)
    trainlst = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/train.lst', 'w')
    print('train')
    for fi in train:
        print(fi)
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        print(gogn.points.shape[0])
        mm = ((min([a[0] for a in gogn.v0]), max([a[0] for a in gogn.v0])),
              (min([a[1] for a in gogn.v0]), max([a[1] for a in gogn.v0])),
              (min([a[2] for a in gogn.v0]), max([a[2] for a in gogn.v0])))
        num_ilat = max(32, int((gogn.points.shape[0] / 1000) ** (1/2)))
        ilat_X = []
        for i in range(num_ilat):
            ilat_X.append([])
            for j in range(num_ilat):
                ilat_X[i].append([])
        ilat_Y = []
        for i in range(num_ilat):
            ilat_Y.append([])
            for j in range(num_ilat):
                ilat_Y[i].append([])
        ilat_Z = []
        for i in range(num_ilat):
            ilat_Z.append([])
            for j in range(num_ilat):
                ilat_Z[i].append([])
        ilatid = lambda x, y, z: (max(0, min(int((x - mm[0][0]) * num_ilat / (mm[0][1] - mm[0][0])), num_ilat - 1)),
                                  max(0, min(int((y - mm[1][0]) * num_ilat / (mm[1][1] - mm[1][0])), num_ilat - 1)),
                                  max(0, min(int((z - mm[2][0]) * num_ilat / (mm[2][1] - mm[2][0])), num_ilat - 1)))
        utskra = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/' + fi + '.npz', 'wb')
        vol_points = []
        vol_label = []
        for a in gogn.points:
            minid = ilatid(min([a[0], a[3], a[6]]), min([a[1], a[4], a[7]]), min([a[2], a[5], a[8]]))
            maxid = ilatid(max([a[0], a[3], a[6]]), max([a[1], a[4], a[7]]), max([a[2], a[5], a[8]]))
            for i in range(minid[0], maxid[0] + 1):
                for j in range(minid[1], maxid[1] + 1):
                    ilat_Z[i][j].append(a)
            for i in range(minid[0], maxid[0] + 1):
                for k in range(minid[2], maxid[2] + 1):
                    ilat_Y[i][k].append(a)
            for j in range(minid[1], maxid[1] + 1):
                for k in range(minid[2], maxid[2] + 1):
                    ilat_X[j][k].append(a)
        for i in range(4096):
            randpoint = (np.random.uniform(mm[0][0], mm[0][1]),
                         np.random.uniform(mm[1][0], mm[1][1]),
                         np.random.uniform(mm[2][0], mm[2][1]))
            # Is point inside or outside shape?
            fata = ilatid(randpoint[0], randpoint[1], randpoint[2])
            pct = [0, 0]
            if len(ilat_X[fata[1]][fata[2]]) < len(ilat_Y[fata[0]][fata[2]]) and \
               len(ilat_X[fata[1]][fata[2]]) < len(ilat_Z[fata[0]][fata[1]]):
                for a in ilat_X[fata[1]][fata[2]]:
                    isi = isInside(a, randpoint, 'x')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            elif len(ilat_Y[fata[0]][fata[2]]) < len(ilat_X[fata[1]][fata[2]]) and \
                 len(ilat_Y[fata[0]][fata[2]]) < len(ilat_Z[fata[0]][fata[1]]):
                for a in ilat_Y[fata[0]][fata[2]]:
                    isi = isInside(a, randpoint, 'y')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            elif len(ilat_Z[fata[0]][fata[1]]) < len(ilat_X[fata[1]][fata[2]]) and \
                 len(ilat_Z[fata[0]][fata[1]]) < len(ilat_Y[fata[0]][fata[2]]):
                for a in ilat_Y[fata[0]][fata[2]]:
                    isi = isInside(a, randpoint, 'z')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                vol_points.append(randpoint)
                vol_label.append(pct[0] % 2)
        print('train vol', time.time() - byrjun)
        near_points = []
        near_label = []
        for i in range(4096):
            gp = gogn.points[np.random.randint(0, gogn.points.shape[0])]
            randpoint = (gp[0] + np.random.uniform(mm[0][0] / 100, mm[0][1] / 100),
                         gp[1] + np.random.uniform(mm[1][0] / 100, mm[1][1] / 100),
                         gp[2] + np.random.uniform(mm[2][0] / 100, mm[2][1] / 100))
            # Is point inside or outside shape?
            fata = ilatid(randpoint[0], randpoint[1], randpoint[2])
            pct = [0, 0]
            if len(ilat_X[fata[1]][fata[2]]) < len(ilat_Y[fata[0]][fata[2]]) and \
               len(ilat_X[fata[1]][fata[2]]) < len(ilat_Z[fata[0]][fata[1]]):
                for a in ilat_X[fata[1]][fata[2]]:
                    isi = isInside(a, randpoint, 'x')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            elif len(ilat_Y[fata[0]][fata[2]]) < len(ilat_X[fata[1]][fata[2]]) and \
                 len(ilat_Y[fata[0]][fata[2]]) < len(ilat_Z[fata[0]][fata[1]]):
                for a in ilat_Y[fata[0]][fata[2]]:
                    isi = isInside(a, randpoint, 'y')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            elif len(ilat_Z[fata[0]][fata[1]]) < len(ilat_X[fata[1]][fata[2]]) and \
                 len(ilat_Z[fata[0]][fata[1]]) < len(ilat_Y[fata[0]][fata[2]]):
                for a in ilat_Y[fata[0]][fata[2]]:
                    isi = isInside(a, randpoint, 'z')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                near_points.append(randpoint)
                near_label.append(pct[0] % 2)
        print('train near', time.time() - byrjun)
        if len(vol_points) > 0 and len(near_points) > 0:
            trainlst.write(fi + '.npz\n')
            np.savez(utskra,
                     vol_points=np.array(vol_points, dtype=np.float32),
                     vol_label=np.array(vol_label),
                     near_points=np.array(near_points, dtype=np.float32),
                     near_label=np.array(near_label))
    trainlst.close()
    vallst = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/val.lst', 'w')
    print('val')
    for fi in val:
        print(fi)
        vallst.write(fi + '.npz\n')
        gogn = mesh.Mesh.from_file(mappa + 'STLdata/' + cat + '_' + fi)
        print(gogn.points.shape[0])
        mm = ((min([a[0] for a in gogn.v0]), max([a[0] for a in gogn.v0])),
              (min([a[1] for a in gogn.v0]), max([a[1] for a in gogn.v0])),
              (min([a[2] for a in gogn.v0]), max([a[2] for a in gogn.v0])))
        num_ilat = max(32, int((gogn.points.shape[0] / 1000) ** (1/2)))
        ilat_X = []
        for i in range(num_ilat):
            ilat_X.append([])
            for j in range(num_ilat):
                ilat_X[i].append([])
        ilat_Y = []
        for i in range(num_ilat):
            ilat_Y.append([])
            for j in range(num_ilat):
                ilat_Y[i].append([])
        ilat_Z = []
        for i in range(num_ilat):
            ilat_Z.append([])
            for j in range(num_ilat):
                ilat_Z[i].append([])
        ilatid = lambda x, y, z: (max(0, min(int((x - mm[0][0]) * num_ilat / (mm[0][1] - mm[0][0])), num_ilat - 1)),
                                  max(0, min(int((y - mm[1][0]) * num_ilat / (mm[1][1] - mm[1][0])), num_ilat - 1)),
                                  max(0, min(int((z - mm[2][0]) * num_ilat / (mm[2][1] - mm[2][0])), num_ilat - 1)))
        utskra = open(mappa + '3DShape2VecSet/data/ShapeNetV2_point/' + cat + '/' + fi + '.npz', 'wb')
        vol_points = []
        vol_label = []
        for a in gogn.points:
            minid = ilatid(min([a[0], a[3], a[6]]), min([a[1], a[4], a[7]]), min([a[2], a[5], a[8]]))
            maxid = ilatid(max([a[0], a[3], a[6]]), max([a[1], a[4], a[7]]), max([a[2], a[5], a[8]]))
            for i in range(minid[0], maxid[0] + 1):
                for j in range(minid[1], maxid[1] + 1):
                    ilat_Z[i][j].append(a)
            for i in range(minid[0], maxid[0] + 1):
                for k in range(minid[2], maxid[2] + 1):
                    ilat_Y[i][k].append(a)
            for j in range(minid[1], maxid[1] + 1):
                for k in range(minid[2], maxid[2] + 1):
                    ilat_X[j][k].append(a)
        for i in range(4096):
            randpoint = (np.random.uniform(mm[0][0], mm[0][1]),
                         np.random.uniform(mm[1][0], mm[1][1]),
                         np.random.uniform(mm[2][0], mm[2][1]))
            # Is point inside or outside shape?
            fata = ilatid(randpoint[0], randpoint[1], randpoint[2])
            pct = [0, 0]
            if len(ilat_X[fata[1]][fata[2]]) < len(ilat_Y[fata[0]][fata[2]]) and \
               len(ilat_X[fata[1]][fata[2]]) < len(ilat_Z[fata[0]][fata[1]]):
                for a in ilat_X[fata[1]][fata[2]]:
                    isi = isInside(a, randpoint, 'x')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            elif len(ilat_Y[fata[0]][fata[2]]) < len(ilat_X[fata[1]][fata[2]]) and \
                 len(ilat_Y[fata[0]][fata[2]]) < len(ilat_Z[fata[0]][fata[1]]):
                for a in ilat_Y[fata[0]][fata[2]]:
                    isi = isInside(a, randpoint, 'y')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            elif len(ilat_Z[fata[0]][fata[1]]) < len(ilat_X[fata[1]][fata[2]]) and \
                 len(ilat_Z[fata[0]][fata[1]]) < len(ilat_Y[fata[0]][fata[2]]):
                for a in ilat_Y[fata[0]][fata[2]]:
                    isi = isInside(a, randpoint, 'z')
                    if isi == 1:
                        pct[1] += 1
                    elif isi == -1:
                        pct[0] += 1
            if len(pct) == 2 and pct[0] % 2 == pct[1] % 2:
                vol_points.append(randpoint)
                vol_label.append(pct[0] % 2)
        if len(vol_points) > 0:
            print('val vol', time.time() - byrjun)
            np.savez(utskra,
                     vol_points=np.array(vol_points, dtype=np.float32),
                     vol_label=np.array(vol_label))
    vallst.close()
    print(time.time() - byrjun)
    print('----')

body parts


mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/body parts: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/body parts: File exists


train
3D ear.stl
222732
train vol 6.58174204826355
train near 9.282470226287842
We Are Beautiful – 662767-FSAN-1 – surface.stl
425528
train vol 17.76410222053528
train near 19.414664268493652
Female lymph node right.stl
254663
train vol 26.694612979888916
train near 31.313730001449585
We Are Beautiful – 662767-FSAN-1 – solid.stl
487252
train vol 40.89013624191284
train near 43.84591102600098
We Are Beautiful – 426060-FSAN-1 – surface.stl
244908
train vol 49.19883632659912
train near 50.6528902053833
Vh-m-blood-vasculature.stl
416538
train vol 57.07714104652405
train near 65.31736922264099
We Are Beautiful – 591522-FSAN-3 – surface.stl
130469
train vol 68.28385996818542
train near 69.3153510093689
Orbit of the Face, STL file processed 1.0.0.stl
517986
train vol 80.80561423301697
train near 82.1377820968628
Vh-f-mammary-gland-l.stl
270799
train vol 87.06798124313354
train near 93.70339107513428
We Are Beautiful – 591522-FSAN-2 – solid.stl
386810
train vol 101.39721918106079
train near 10

mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/buildings: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/buildings: File exists


train
White House.stl
2960
train vol 3.061429023742676
train near 3.470147132873535
Castellum Ras el Oued Gordab, 1904.stl
3196
train vol 4.463785886764526
train near 6.621844053268433
3D-Modell dorische Ordnung 20220608.stl
5601580
train vol 77.74757409095764
train near 175.03686499595642
Shanghai Tower.stl
26896
train vol 176.20807600021362
train near 179.1703941822052
Space Needle.stl
3160
train vol 179.99450016021729
train near 182.63355803489685
Lone-monk cycles and exposure-node demo (Blender demo 2.92).stl
5605036
train vol 251.42593502998352


...

train
Tree of weak orderings in concertina cube.stl
12
train vol 2.4130680561065674
train near 2.7807631492614746
LowPoly Sphere 07.stl
52
train vol 3.161029100418091
train near 3.8653512001037598
CP bz 1.stl
24
train vol 4.00881028175354
train near 4.071106195449829
Inverse-reuleaux-tetrahedron-1-intersection.stl
15160
train vol 4.718508005142212
train near 5.376993179321289
Hennig-reuleaux-tetrahedron.stl
1532
train vol 5.50301718711853
train near 5.812821388244629
LowPoly Sphere 12.stl
140
train vol 6.147418260574341
train near 7.022718191146851
LowPoly Sphere 09.stl
86
train vol 7.429893255233765
train near 8.113263130187988
Sphere 3D.stl
760
train vol 8.39474606513977
train near 9.421788215637207
Soccer ball.stl
16380
train vol 10.338529109954834
train near 12.52163028717041
Fcc bz 2.stl
220
train vol 12.809229135513306
train near 13.397144079208374
Spectrahedron.stl
12100
train vol 14.020684003829956
train near 14.921826124191284
Bcc bz 2.stl
77
train vol 16.06414532661438
train 

mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/objects in space: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/objects in space: File exists


exception (False, 'No lines found, impossible to read')
train
Kepler Space Telescope.stl
170332
train vol 6.598564147949219
train near 11.68839406967163
Curiosity Rover.stl
114899
train vol 14.227003335952759
train near 18.159698247909546
Galileo without probe 3D.stl
16729
train vol 18.726445198059082
train near 22.92699122428894
New Horizons.stl
160886
train vol 27.19894504547119
train near 34.08537030220032
Soyuz 7K-LOK.stl
157260
train vol 37.006301164627075
train near 53.78975319862366
Voyager Probe.stl
284737
train vol 58.05622315406799
train near 98.8612711429596
Mars Reconnaissance Orbiter 3D Model.stl
14777
train vol 99.36392116546631
train near 102.11986827850342
Galileo stowed 3D.stl
16621
train vol 102.62789821624756
train near 106.72487497329712
Triana Concept 3D.stl
8798
train vol 107.31232523918152
train near 110.29092025756836
HESSI-RHESSI 3D.stl
23388
train vol 110.85543012619019
train near 119.48913621902466
3D-Modell James Webb Space Telescope.stl
7988836
train vol 21

mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_point/sculptures: File exists
mkdir: /Users/002-nathan/Desktop/Envalys/envalys-nathan/3DShape2VecSet/data/ShapeNetV2_watertight/sculptures: File exists


train
庚申塔道標（横浜市旭区今宿南町）.stl
1750000
train vol 34.3658390045166
train near 39.61248183250427
Scan the World - Pietà (Michelangelo).stl
815738
train vol 57.11745500564575
train near 62.980786085128784
庚申塔（須賀神社）.stl
1000000
train vol 82.17565989494324
train near 87.08618307113647
Leyre - Capitel.stl
1323341
train vol 112.76980185508728
train near 117.4665539264679
Wasserbüffel und Knabe - 3D model by noe-3d.at - Sketchfab.stl
661492
train vol 131.06582283973694
train near 143.11545586585999
庚申塔（野木町友沼）.stl
1750000
train vol 172.16325497627258
train near 176.79315996170044
Moai.stl
11176
train vol 177.66335892677307
train near 179.78091883659363
二十三夜塔（耕雲庵）.stl
986079
train vol 200.0027379989624
train near 204.9110450744629
Scan the World - Venus de Milo.stl
607274
train vol 217.3175458908081
train near 222.767098903656
Danaid NMSk 1854 (Auguste Rodin) - Nationalmuseum - 76c5c234c6074b13a94bf793c276a509.stl
110538
train vol 226.24468088150024
train near 227.9327530860901
The Age Of Bronze (Au

Sniðbreytingin tók eina klukkustund.

## Líkan
Líkanið þarf ekki að búa til góðar myndir. Það þarf bara að virka án villna, sem sýnir okkur að við þurfum bara meiri og betri gögn, því vísindagreinin er með góðar myndir.

In [18]:
os.system('cd ' + mappa)
os.system('torchrun --nproc_per_node=4 3DShape2VecSet/main_ae.py --accum_iter=2 --model ae_d512_m512 '
          '--data_path 3DShape2VecSet/data --output_dir output/ae/ae_d512_m512 --log_dir output/ae/ae_d512_m512 '
          '--num_workers 10 --point_cloud_size 2048 --batch_size 64 --epochs 200 --warmup_epochs 5')

W0825 09:05:21.680789 8286703424 torch/distributed/elastic/multiprocessing/redirects.py:28] NOTE: Redirects are currently not supported in Windows or MacOs.
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] 
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] *****************************************
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0825 09:05:21.693071 8286703424 torch/distributed/run.py:779] *****************************************


['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']

['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
['body parts', 'buildings', 'geometric shapes', 'objects in space', 'sculptures']
Model = AutoEncoder(
  (cross_attend_blocks): ModuleList(
    (0): PreNorm(
      (fn): Attention(
        (to_q): Linear(in_features=512, out_features=512, bias=False)
        (to_kv): Linear(in_features=512, out_features=1024, bias=False)
        (to_out): Linear(in_features=512, out_features=512, bias=True)
        (drop_path): Identity()


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Epoch: [0]  [ 0/47]  eta: 0:43:45  lr: 0.000000  loss: 0.7250 (0.7250)  loss_vol: 0.6592 (0.6592)  loss_near: 0.6580 (0.6580)  iou: 0.0139 (0.0139)  time: 55.8586  data: 19.8939
Epoch: [0]  [ 0/47]  eta: 0:49:22  lr: 0.000000  loss: 0.7185 (0.7185)  loss_vol: 0.6532 (0.6532)  loss_near: 0.6534 (0.6534)  iou: 0.0089 (0.0089)  time: 63.0310  data: 19.9302
Epoch: [0]  [ 0/47]  eta: 0:50:54  lr: 0.000000  loss: 0.7204 (0.7204)  loss_vol: 0.6549 (0.6549)  loss_near: 0.6550 (0.6550)  iou: 0.0234 (0.0234)  time: 64.9863  data: 16.6555
Epoch: [0]  [ 0/47]  eta: 1:23:32  lr: 0.000000  loss: 0.7272 (0.7272)  loss_vol: 0.6611 (0.6611)  loss_near: 0.6610 (0.6610)  iou: 0.0238 (0.0238)  time: 106.6386  data: 16.7561


W0825 09:35:12.375863 8286703424 torch/distributed/elastic/agent/server/api.py:688] Received Signals.SIGINT death signal, shutting down workers
W0825 09:35:12.392325 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40348 closing signal SIGINT
W0825 09:35:12.393383 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40349 closing signal SIGINT
W0825 09:35:12.393764 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40350 closing signal SIGINT
W0825 09:35:12.398679 8286703424 torch/distributed/elastic/multiprocessing/api.py:858] Sending process 40351 closing signal SIGINT
W0825 09:35:42.408644 8286703424 torch/distributed/elastic/multiprocessing/api.py:875] Unable to shutdown process 40348 via Signals.SIGINT, forcefully exiting via Signals.SIGKILL
W0825 09:35:49.303166 8286703424 torch/distributed/elastic/multiprocessing/api.py:875] Unable to shutdown process 40349 via Signals.SIGINT, forcefully ex

256

Það eru 200 lotna (e. *epochs*), og hver lota tekur klukkustund; við eigum ekki nógan tíma fyrir alla þjálfunina. **En hún er hafin án villna.**

# Lokaorð
Hvert förum við héðan? 3DShape2VecSet er með margar takmarkanir.
- Það kann bara að búa til einfaldar myndir, eins og "stóll". Með greiningarlíkani getum við mögulega búið til fjölfaldar myndir. Ef notandi segir "stóll með kodda", þá býr líkanið okkar til mynd af stóli og mynd af kodda, sem líkanið sameinar.
- Það skilur ekki mannamál.
- Það er ekki með mikil gögn. Með mínu forriti hérna getum við búið til eins mikil gögn og við viljum; við getum tekið myndir frá notendum og búum til lista yfir punkta, og notað þá til að þjálfa líkanið.

Ég er viss um að, með meiri gögn og betri tölvu, við getum að minnsta kosti eitt af þessum.